# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Lauren Michel

**ID**: lam425

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/BEE4750/hw5-lmic`
Precompiling project...
  ✓ Latexify → DataFramesExt
  1 dependency successfully precompiled in 3 seconds. 232 already precompiled.


In [3]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [28]:
waste_model = Model(HiGHS.Optimizer)

@variable(waste_model, W[1:3,1:3] >= 0)
@variable(waste_model, R[1:3,1:3] >= 0)
@variable(waste_model, Y[1:3], Bin)

@objective(waste_model, Min, (2500*Y[1]) + (1500*Y[2]) + (2000*Y[3]) + (82.5*W[1,1]) + (75*W[2,1]) + (90*W[3,1]) + (68*W[1,2]) + (60.5*W[2,2]) + (90.5*W[3,2]) + (57.5*W[1,3]) + (72.5*W[2,3]) + (69.5*W[3,3]) + (57.5*R[1,3]) + (72.5*R[2,3]) + (69.5*R[3,3]) + (75*R[2,1]) + (90*R[3,1]))

@constraint(waste_model, commit1, W[1,1]+W[2,1]+W[3,1]+R[2,1]+R[3,1] <= 1000000*Y[1])
@constraint(waste_model, commit2, W[2,1]+W[2,2]+W[2,3] <= 1000000*Y[2])
@constraint(waste_model, commit3, Y[3] == 1)

@constraint(waste_model, massbal1, W[1,1]+W[1,2]+W[1,3] == 100)
@constraint(waste_model, massbal2, W[2,1]+W[2,2]+W[2,3] == 90)
@constraint(waste_model, massbal3, W[3,1]+W[3,2]+W[3,3] == 120)

@constraint(waste_model, wteres, (0.16*(W[1,1]+W[2,1]+W[3,1]+R[2,1]+R[3,1]))-R[1,3] == 0)
@constraint(waste_model, recmrfres, (0.14*0.4*(W[1,2]+W[2,2]+W[3,2]))-R[2,3] == 0)
@constraint(waste_model, mrfres, (0.6*(W[1,2]+W[2,2]+W[3,2]))-R[2,1]-R[2,3] == 0)

@constraint(waste_model, wtedisp, W[1,1]+W[2,1]+W[2,3]+R[2,1]+R[3,1] <= 210)
@constraint(waste_model, mrfdisp, W[1,2]+W[2,2]+W[3,2] <= 350)
@constraint(waste_model, lfdisp, W[1,3]+W[2,3]+W[3,3]+R[1,3]+R[2,3]+R[3,3] <= 200)

print(waste_model)

Min 2500 Y[1] + 1500 Y[2] + 2000 Y[3] + 82.5 W[1,1] + 75 W[2,1] + 90 W[3,1] + 68 W[1,2] + 60.5 W[2,2] + 90.5 W[3,2] + 57.5 W[1,3] + 72.5 W[2,3] + 69.5 W[3,3] + 57.5 R[1,3] + 72.5 R[2,3] + 69.5 R[3,3] + 75 R[2,1] + 90 R[3,1]
Subject to
 commit3 : Y[3] = 1
 massbal1 : W[1,1] + W[1,2] + W[1,3] = 100
 massbal2 : W[2,1] + W[2,2] + W[2,3] = 90
 massbal3 : W[3,1] + W[3,2] + W[3,3] = 120
 wteres : 0.16 W[1,1] + 0.16 W[2,1] + 0.16 W[3,1] + 0.16 R[2,1] + 0.16 R[3,1] - R[1,3] = 0
 recmrfres : 0.05600000000000001 W[1,2] + 0.05600000000000001 W[2,2] + 0.05600000000000001 W[3,2] - R[2,3] = 0
 mrfres : 0.6 W[1,2] + 0.6 W[2,2] + 0.6 W[3,2] - R[2,1] - R[2,3] = 0
 commit1 : W[1,1] + W[2,1] + W[3,1] + R[2,1] + R[3,1] - 1000000 Y[1] ≤ 0
 commit2 : W[2,1] + W[2,2] + W[2,3] - 1000000 Y[2] ≤ 0
 wtedisp : W[1,1] + W[2,1] + W[2,3] + R[2,1] + R[3,1] ≤ 210
 mrfdisp : W[1,2] + W[2,2] + W[3,2] ≤ 350
 lfdisp : W[1,3] + W[2,3] + W[3,3] + R[1,3] + R[2,3] + R[3,3] ≤ 200
 W[1,1] ≥ 0
 W[2,1] ≥ 0
 W[3,1] ≥ 0
 W[1,2] ≥ 0


In [29]:
optimize!(waste_model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [6e-02, 1e+06]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 4e+02]
Presolving model
10 rows, 15 cols, 44 nonzeros  0s
8 rows, 13 cols, 39 nonzeros  0s
6 rows, 9 cols, 25 nonzeros  0s
5 rows, 9 cols, 19 nonzeros  0s

Solving MIP model with:
   5 rows
   9 cols (0 binary, 0 integer, 0 implied int., 9 continuous)
   19 nonzeros
MIP-Timing:      0.0007 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol     

In [30]:
@show value.(W)

value.(W) = [0.0 0.0 100.0; 90.0 0.0 -0.0; 40.95238095238096 0.0 79.04761904761904]


3×3 Matrix{Float64}:
  0.0     0.0  100.0
 90.0     0.0   -0.0
 40.9524  0.0   79.0476

In [33]:
@show value.(W[1,3])

value.(W[1, 3]) = 100.0


100.0

In [31]:
@show value.(R)

value.(R) = [0.0 0.0 20.952380952380956; 0.0 0.0 0.0; 0.0 0.0 0.0]


3×3 Matrix{Float64}:
 0.0  0.0  20.9524
 0.0  0.0   0.0
 0.0  0.0   0.0

In [32]:
@show value.(Y)

value.(Y) = [1.0, 1.0, 1.0]


3-element Vector{Float64}:
 1.0
 1.0
 1.0

## References

List any external references consulted, including classmates.